# Individual Planning Report: Predicting Video Game Server Usage

**Date:** November 2025  
**Course:** Data Science Project

This report analyzes player and session data from a MineCraft research server to address predictive questions about player behavior and server usage patterns.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## XLSX to CSV Transformation

First, we convert the Excel files to CSV format for easier processing.

In [ ]:
# Read the sessions xlsx file
sessions_df = pd.read_excel('sessions (2).xlsx')

# Display the first few rows
print(f"Sessions data shape: {sessions_df.shape}")
sessions_df.head()

In [ ]:
# Convert to CSV
sessions_df.to_csv('sessions.csv', index=False)
print("✓ sessions.csv created successfully")

In [ ]:
# Read the players xlsx file
players_df = pd.read_excel('players.xlsx')

# Display the first few rows
print(f"Players data shape: {players_df.shape}")
players_df.head()

In [ ]:
# Convert to CSV
players_df.to_csv('players.csv', index=False)
print("✓ players.csv created successfully")

In [ ]:
# Verify the CSV files were created
import os
csv_files = ['sessions.csv', 'players.csv']
for csv_file in csv_files:
    if os.path.exists(csv_file):
        size = os.path.getsize(csv_file)
        print(f"✓ {csv_file} - {size:,} bytes")
    else:
        print(f"✗ {csv_file} - not found")

---

## 1. Data Description

This section provides a comprehensive analysis of the MineCraft research server dataset, including player profiles and session logs.

### 1.1 Loading the Datasets

Loading both CSV files that were previously converted from Excel format.

In [ ]:
# Reload the datasets from CSV files
players_df = pd.read_csv('players.csv')
sessions_df = pd.read_csv('sessions.csv')

print("Data successfully loaded!")
print(f"Players dataset: {players_df.shape[0]} rows, {players_df.shape[1]} columns")
print(f"Sessions dataset: {sessions_df.shape[0]} rows, {sessions_df.shape[1]} columns")

### 1.2 Players Dataset - Variable Description and Structure

In [ ]:
# Display basic information about players dataset
print("PLAYERS DATASET STRUCTURE")
print("="*50)
players_df.info()
print("\n")

# Create a comprehensive variable description table for players
players_vars = pd.DataFrame({
    'Variable': players_df.columns,
    'Type': players_df.dtypes.astype(str).values,
    'Non-Null Count': players_df.count().values,
    'Null Count': players_df.isnull().sum().values,
    'Unique Values': [players_df[col].nunique() for col in players_df.columns],
    'Description': [
        'Player experience level (Amateur/Regular/Pro/Veteran)',
        'Newsletter subscription status (True/False)',
        'Hashed email identifier for privacy',
        'Total hours played on the server',
        'Player name (pseudonym)',
        'Player gender (Male/Female)',
        'Player age in years'
    ]
})

print("\nPLAYERS DATASET - VARIABLE DESCRIPTIONS")
print("="*80)
players_vars

### 1.3 Sessions Dataset - Variable Description and Structure

In [ ]:
# Display basic information about sessions dataset
print("SESSIONS DATASET STRUCTURE")
print("="*50)
sessions_df.info()
print("\n")

# Create a comprehensive variable description table for sessions
sessions_vars = pd.DataFrame({
    'Variable': sessions_df.columns,
    'Type': sessions_df.dtypes.astype(str).values,
    'Non-Null Count': sessions_df.count().values,
    'Null Count': sessions_df.isnull().sum().values,
    'Unique Values': [sessions_df[col].nunique() for col in sessions_df.columns],
    'Description': [
        'Hashed email identifier linking to players table',
        'Session start time (DD/MM/YYYY HH:MM format)',
        'Session end time (DD/MM/YYYY HH:MM format)',
        'Original start timestamp (Unix epoch in milliseconds)',
        'Original end timestamp (Unix epoch in milliseconds)'
    ]
})

print("SESSIONS DATASET - VARIABLE DESCRIPTIONS")
print("="*80)
sessions_vars

### 1.4 Descriptive Statistics - Players Dataset

In [ ]:
# Numerical variables summary statistics
print("PLAYERS DATASET - NUMERICAL VARIABLES SUMMARY")
print("="*60)
numerical_stats = players_df[['played_hours', 'Age']].describe()
numerical_stats.loc['missing'] = players_df[['played_hours', 'Age']].isnull().sum()
print(numerical_stats.round(2))

In [ ]:
# Categorical variables distribution
print("\nPLAYERS DATASET - CATEGORICAL VARIABLES DISTRIBUTION")
print("="*60)

categorical_cols = ['experience', 'subscribe', 'gender']
for col in categorical_cols:
    print(f"\n{col.upper()} Distribution:")
    value_counts = players_df[col].value_counts()
    percentage = players_df[col].value_counts(normalize=True) * 100
    cat_summary = pd.DataFrame({
        'Count': value_counts,
        'Percentage': percentage.round(2)
    })
    print(cat_summary)

### 1.5 Mean Values Table for Quantitative Variables (Required by Assignment)

In [ ]:
# Compute mean values for quantitative variables in players dataset
quantitative_vars = players_df.select_dtypes(include=[np.number]).columns.tolist()

mean_values = pd.DataFrame({
    'Variable': quantitative_vars,
    'Mean Value': [players_df[col].mean() for col in quantitative_vars],
    'Standard Deviation': [players_df[col].std() for col in quantitative_vars],
    'Valid Count': [players_df[col].count() for col in quantitative_vars],
    'Missing Count': [players_df[col].isnull().sum() for col in quantitative_vars]
})

print("MEAN VALUES FOR QUANTITATIVE VARIABLES IN PLAYERS DATASET")
print("="*60)
mean_values

### 1.6 Data Quality Assessment

In [ ]:
# Check for missing values
print("DATA QUALITY ISSUES")
print("="*40)
print("\n1. MISSING VALUES:")
print("\nPlayers Dataset:")
players_missing = players_df.isnull().sum()
if players_missing.sum() > 0:
    print(players_missing[players_missing > 0])
else:
    print("No missing values found")

print("\nSessions Dataset:")
sessions_missing = sessions_df[['hashedEmail', 'start_time', 'end_time', 'original_start_time', 'original_end_time']].isnull().sum()
if sessions_missing.sum() > 0:
    print(sessions_missing[sessions_missing > 0])
else:
    print("No missing values found")

In [ ]:
# Check data consistency between tables
print("\n2. DATA CONSISTENCY:")
players_emails = set(players_df['hashedEmail'])
sessions_emails = set(sessions_df['hashedEmail'])

print(f"Players in players.csv: {len(players_emails)}")
print(f"Unique players in sessions.csv: {len(sessions_emails)}")
print(f"Players with sessions: {len(players_emails.intersection(sessions_emails))}")
print(f"Players without sessions: {len(players_emails - sessions_emails)}")
print(f"Sessions without player records: {len(sessions_emails - players_emails)}")

---

## 2. Data Wrangling and Cleaning

This section performs the minimum necessary data wrangling to convert the data into tidy format and address quality issues identified in the exploration phase.

### 2.1 Data Type Optimization and Conversion

In [ ]:
# Reload data to ensure clean state
players_df = pd.read_csv('players.csv')
sessions_df = pd.read_csv('sessions.csv')

print("Original data types:")
print("\nPlayers dataset:")
print(players_df.dtypes)
print(f"Shape: {players_df.shape}")

print("\nSessions dataset:")
print(sessions_df.dtypes) 
print(f"Shape: {sessions_df.shape}")

In [ ]:
# Convert categorical variables to proper categories for efficiency
print("Converting categorical variables to category dtype...")

# Convert players categorical columns
players_df['experience'] = players_df['experience'].astype('category')
players_df['gender'] = players_df['gender'].astype('category')
players_df['subscribe'] = players_df['subscribe'].astype(bool)

# Convert timestamp columns in sessions
print("Converting timestamp columns to datetime...")
sessions_df['start_time'] = pd.to_datetime(sessions_df['start_time'], format='%d/%m/%Y %H:%M')
sessions_df['end_time'] = pd.to_datetime(sessions_df['end_time'], format='%d/%m/%Y %H:%M')

print("\nOptimized data types:")
print("\nPlayers dataset:")
print(players_df.dtypes)

print("\nSessions dataset:")
print(sessions_df.dtypes)

### 2.2 Feature Engineering and Derived Variables

In [ ]:
# Calculate session duration in minutes
sessions_df['session_duration_minutes'] = (sessions_df['end_time'] - sessions_df['start_time']).dt.total_seconds() / 60

# Create time-based features
sessions_df['start_hour'] = sessions_df['start_time'].dt.hour
sessions_df['start_day_of_week'] = sessions_df['start_time'].dt.day_name()
sessions_df['start_month'] = sessions_df['start_time'].dt.month

# Categorize session times
def categorize_time(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Night'

sessions_df['time_period'] = sessions_df['start_hour'].apply(categorize_time)
sessions_df['time_period'] = sessions_df['time_period'].astype('category')

print("Session duration statistics (minutes):")
print(sessions_df['session_duration_minutes'].describe())
print(f"\nNegative durations (data quality issue): {(sessions_df['session_duration_minutes'] < 0).sum()}")
print(f"Zero duration sessions: {(sessions_df['session_duration_minutes'] == 0).sum()}")

print("\nTime period distribution:")
print(sessions_df['time_period'].value_counts())

### 2.3 Player-Level Aggregations

In [ ]:
# Create player-level summary statistics from sessions
player_session_stats = sessions_df.groupby('hashedEmail').agg({
    'session_duration_minutes': ['count', 'sum', 'mean', 'std'],
    'start_time': ['min', 'max']
}).round(2)

# Flatten column names
player_session_stats.columns = ['_'.join(col).strip() for col in player_session_stats.columns.values]
player_session_stats.rename(columns={
    'session_duration_minutes_count': 'total_sessions',
    'session_duration_minutes_sum': 'total_playtime_minutes',
    'session_duration_minutes_mean': 'avg_session_duration',
    'session_duration_minutes_std': 'session_duration_std',
    'start_time_min': 'first_session',
    'start_time_max': 'last_session'
}, inplace=True)

# Calculate days between first and last session
player_session_stats['engagement_days'] = (
    player_session_stats['last_session'] - player_session_stats['first_session']
).dt.days + 1

# Fill NaN std with 0 for players with only one session
player_session_stats['session_duration_std'].fillna(0, inplace=True)

print(f"Player session statistics shape: {player_session_stats.shape}")
print("\nPlayer session statistics summary:")
print(player_session_stats.describe())

### 2.4 Data Integration and Tidy Format

In [ ]:
# Merge players data with session statistics
# Use left join to keep all players (even those without sessions)
players_complete = players_df.merge(player_session_stats, 
                                   left_on='hashedEmail', 
                                   right_index=True, 
                                   how='left')

# Fill missing values for players without sessions
session_cols = ['total_sessions', 'total_playtime_minutes', 'avg_session_duration', 
                'session_duration_std', 'engagement_days']
for col in session_cols:
    if col in players_complete.columns:
        players_complete[col].fillna(0, inplace=True)

# Create engagement categories based on total sessions
def categorize_engagement(total_sessions):
    if total_sessions == 0:
        return 'No Activity'
    elif total_sessions <= 5:
        return 'Low'
    elif total_sessions <= 20:
        return 'Medium'
    else:
        return 'High'

players_complete['engagement_level'] = players_complete['total_sessions'].apply(categorize_engagement)
players_complete['engagement_level'] = players_complete['engagement_level'].astype('category')

print(f"Complete dataset shape: {players_complete.shape}")
print(f"Players without sessions: {players_complete['total_sessions'].eq(0).sum()}")
print("\nEngagement level distribution:")
print(players_complete['engagement_level'].value_counts())

### 2.5 Data Quality Issues Documentation

In [ ]:
# Document all data quality issues found during wrangling
print("DATA QUALITY ASSESSMENT SUMMARY")
print("=" * 50)

print("\n1. MISSING VALUES:")
print(f"   - Age missing in players dataset: {players_df['Age'].isnull().sum()} records")
print(f"   - No missing values in sessions dataset")

print("\n2. DATA CONSISTENCY:")
print(f"   - Players in players.csv: {len(players_df)}")
print(f"   - Players with sessions: {len(player_session_stats)}")
print(f"   - Players without sessions: {len(players_df) - len(player_session_stats)}")

print("\n3. DATA RANGE VALIDATION:")
print(f"   - Age range: {players_df['Age'].min():.0f} to {players_df['Age'].max():.0f} years")
print(f"   - Played hours range: {players_df['played_hours'].min():.1f} to {players_df['played_hours'].max():.1f} hours")
print(f"   - Session duration range: {sessions_df['session_duration_minutes'].min():.1f} to {sessions_df['session_duration_minutes'].max():.1f} minutes")

print("\n4. POTENTIAL OUTLIERS:")
outliers_age = players_df[(players_df['Age'] < 10) | (players_df['Age'] > 60)]['Age'].count()
outliers_hours = players_df[players_df['played_hours'] > 100]['played_hours'].count()
outliers_session = sessions_df[sessions_df['session_duration_minutes'] > 300]['session_duration_minutes'].count()

print(f"   - Age outliers (< 10 or > 60): {outliers_age}")
print(f"   - High playtime outliers (> 100 hours): {outliers_hours}")
print(f"   - Long session outliers (> 5 hours): {outliers_session}")

print("\n5. DATA INTEGRITY:")
print(f"   - Duplicate player records: {players_df['hashedEmail'].duplicated().sum()}")
print(f"   - Duplicate session records: {sessions_df.duplicated().sum()}")
print(f"   - Sessions with negative duration: {(sessions_df['session_duration_minutes'] < 0).sum()}")

### 2.6 Final Cleaned Dataset Summary

In [ ]:
# Display final cleaned datasets
print("FINAL CLEANED DATASETS")
print("=" * 50)

print(f"\nComplete Players Dataset: {players_complete.shape}")
print("Columns:", list(players_complete.columns))
print("\nFirst 3 rows of complete dataset:")
display(players_complete.head(3))

print(f"\nSessions Dataset: {sessions_df.shape}")  
print("Columns:", list(sessions_df.columns))
print("\nSample of sessions data:")
display(sessions_df[['hashedEmail', 'session_duration_minutes', 'time_period', 'start_day_of_week']].head(3))

print("\nData is now in tidy format and ready for analysis!")
print("Key transformations completed:")
print("✓ Categorical variables converted to proper types")
print("✓ Timestamp data converted to datetime")
print("✓ Session durations calculated")
print("✓ Player-level aggregations created")
print("✓ Missing values handled appropriately")
print("✓ Engagement categories defined")
print("✓ Time-based features engineered")